In [ ]:
import numpy as np
import pandas as pd

import glob

import os
import datetime

#from datetime import datetime as dt

from datetime import datetime, timedelta

In [ ]:
pd.set_option('display.max_columns',None)

# HDFC

In [ ]:
HDFC = glob.glob('*PolicyDump*.xlsx', recursive=True)
HDFC

In [ ]:
HDFC_Dataframe = pd.DataFrame()


for i in HDFC:
    Dataframe = pd.read_excel(i,dtype={("Previous Policy No","Policy No"):str}) #,parse_dates=['Pol Issue Date'])
    
    HDFC_Dataframe = pd.concat([HDFC_Dataframe,Dataframe], ignore_index=True)
    
HDFC_Dataframe.head()

In [ ]:
HDFC_Dataframe['Pol Issue Date'] = pd.to_datetime(HDFC_Dataframe['Pol Issue Date'], errors='coerce')#,infer_datetime_format=True)
HDFC_Dataframe['Pol Issue Date']

In [ ]:
HDFC_Dataframe.shape

### * Condition

In [ ]:
HDFC_Final = HDFC_Dataframe[["Previous Policy No","Policy No","Our Share GWP","Pol Issue Date","Customer Name",
                                   "Business Type","Branch Name","Product Name","POL_MOT_VEH_REGISTRATION_NUM",
                            "POL_MOT_MODEL_NAME","ENGINE NUM","CHASSIS NUM"]].copy()
HDFC_Final

In [ ]:
# Add Insurer

HDFC_Final.insert(4,"Insurer",'HDFC Ergo')
HDFC_Final.insert(10,"Make",'')


HDFC_Final

In [ ]:
HDFC_Final.shape

# Royal

In [ ]:
Royal = glob.glob('*Invictus_TeamRoom*.xlsx', recursive=True)
Royal

In [ ]:
Royal_Dataframe = pd.DataFrame()


for i in Royal:
    Dataframe = pd.read_excel(i,dtype={("Previous Policy No","Policy No"):str}) #,parse_dates=['Entry Date'])
    
    Royal_Dataframe = pd.concat([Royal_Dataframe,Dataframe], ignore_index=True)

Royal_Dataframe.head()

In [ ]:
Royal_Dataframe['Entry Date'] = pd.to_datetime(Royal_Dataframe['Entry Date'], errors='coerce')#,infer_datetime_format=True)
Royal_Dataframe['Entry Date']

In [ ]:
Royal_Dataframe.shape

### * Condition

In [ ]:
# (Royal_Dataframe['Business Type'] == 'Renewal') &

Royal_Condition = Royal_Dataframe[ Royal_Dataframe['Product'] == 'Individual Health']
Royal_Condition

In [ ]:
Royal_Health_Final = Royal_Condition[["Policy No","Our Share of Premium","Entry Date","Client Full Name",
                                      "Business Type","Branch Name","Product"]].copy()
Royal_Health_Final

In [ ]:
# Add Insurer

Royal_Health_Final.insert(0,"old_policy_no.",'')
Royal_Health_Final.insert(4,"Insurer",'Royal Sundaram')
Royal_Health_Final.insert(9,"Registration no#",'')

Royal_Health_Final.insert(10,"Make",'')
Royal_Health_Final.insert(11,"Model",'')
Royal_Health_Final.insert(12,"Engine No.",'')
Royal_Health_Final.insert(13,"Chassis No.",'')


Royal_Health_Final

In [ ]:
Royal_Health_Final.shape

# Combine    "All_Insurer"

In [ ]:
# Star_Final

All_Insurer = np.vstack((HDFC_Final, Royal_Health_Final))
                         
All_Insurer

In [ ]:
All_Insurer_Portal = pd.DataFrame(All_Insurer,columns=["Old Policy No.","Policy No.","Net Premium","Booking/Issued Date",
                                                       "Insurer","Customer Name","Business Type","Location","Product",
                                                       "Registration_no.","Make","Model","Engine No.","Chassis No."])
All_Insurer_Portal

In [ ]:
All_Insurer_Portal['Net Premium'] = pd.to_numeric(All_Insurer_Portal['Net Premium'], errors='coerce')#,infer_datetime_format=True)
All_Insurer_Portal['Net Premium']

In [ ]:
#All_Insurer_Portal.to_csv('Net_Premium_Sum.csv',index=True)

In [ ]:
All_Insurer_Portal.info()

### before manimulating dataframe 
<Center> Net_Premium_Sum

In [ ]:
Net_Premium_Sum = All_Insurer_Portal.groupby('Insurer').agg({'Net Premium': 'sum'})
Net_Premium_Sum

In [ ]:
#All_Insurer_Portal.apply(lambda x: x.duplicated().sum())

In [ ]:
#Net_Premium_Sum.to_excel('Net_Premium_Sum.xlsx',index=True)

In [ ]:
All_Insurer_Portal['Insurer'].value_counts().count()

In [ ]:
All_Insurer_Portal.Insurer.value_counts()

# All_Insurer_Portal   "Old Policy No# Apostrophe "

In [ ]:
# step 01 - replace all the special char and alphabetical word and keep only number

All_Insurer_Portal['Old Policy No.'] = All_Insurer_Portal['Old Policy No.'].str.replace("'", "").fillna(All_Insurer_Portal['Old Policy No.'])
All_Insurer_Portal

In [ ]:
# step 09 - add customer name


All_Insurer_Portal.insert(0,"Previous Policy No.","'" + All_Insurer_Portal['Old Policy No.'].astype(str))
All_Insurer_Portal

# All_Insurer_Portal   "NEW Policy No# Apostrophe"

In [ ]:
# step 01 - replace all the special char and alphabetical word and keep only number

All_Insurer_Portal['Policy No.'] = All_Insurer_Portal['Policy No.'].str.replace("'", "").fillna(All_Insurer_Portal['Policy No.'])
All_Insurer_Portal

In [ ]:
# step 09 - add customer name


All_Insurer_Portal.insert(1,"NEW Policy No# Apostrophe","'" + All_Insurer_Portal['Policy No.'].astype(str))
All_Insurer_Portal

## Move "Customer name column" at 1st position

In [ ]:
Move_name = All_Insurer_Portal.pop('Customer Name')
Move_name.head()

In [ ]:
All_Insurer_Portal.insert(0, 'Customer Name', Move_name)
All_Insurer_Portal

## Move "Register No. column" at 2nd position

In [ ]:
Move_Register = All_Insurer_Portal.pop('Registration_no.')
Move_Register.head()

In [ ]:
All_Insurer_Portal.insert(2, 'Registration_no.', Move_Register)
All_Insurer_Portal

## Move "Product column" at 2nd position

In [ ]:
Move_product = All_Insurer_Portal.pop('Product')
Move_product.head()

In [ ]:
All_Insurer_Portal.insert(0, 'Product category', Move_product)
All_Insurer_Portal

### NEW Policy No# Apostrophe

In [ ]:
Move_NEW_Policy_No_Apostrophe = All_Insurer_Portal.pop('NEW Policy No# Apostrophe')
Move_NEW_Policy_No_Apostrophe.head()

In [ ]:
All_Insurer_Portal.insert(6, 'NEW Policy No# Apostrophe', Move_NEW_Policy_No_Apostrophe)
All_Insurer_Portal

### as it is column created 

In [ ]:
All_Insurer_Portal.insert(0,'old policy no# as it is',All_Insurer_Portal['Previous Policy No.'])
All_Insurer_Portal.insert(1,'Reg no# as it is',All_Insurer_Portal['Registration_no.'])
All_Insurer_Portal

## Remove special char

In [ ]:
All_Insurer_Portal['Previous Policy No.'].replace(regex=True, inplace=True, to_replace=r"[^a-zA-Z0-9 ']", value=r'')
All_Insurer_Portal['Old Policy No.'].replace(regex=True, inplace=True, to_replace=r"[^a-zA-Z0-9 ']", value=r'')

All_Insurer_Portal['Registration_no.'].replace(regex=True, inplace=True, to_replace=r"[^a-zA-Z0-9]", value=r'')

All_Insurer_Portal

In [ ]:
dict = {
            "Previous Policy No.":"PolicyNumber",
            "Registration_no.":"Registration_no.__02",
            "NEW Policy No# Apostrophe":"New_Policy No.",
            "Net Premium":"New_Renewed_Premium",
            "Booking/Issued Date":"New_Booking/Issued Date",
            "Insurer":"New_Insurer",
        }

All_Insurer_Portal.rename(columns=dict,inplace=True)
All_Insurer_Portal

In [ ]:
All_Insurer_Portal['New_Booking/Issued Date'] = pd.to_datetime(All_Insurer_Portal['New_Booking/Issued Date'], errors='coerce')
All_Insurer_Portal['New_Booking/Issued Date']

In [ ]:
All_Insurer_Portal['New_Renewed_Premium'] = pd.to_numeric(All_Insurer_Portal['New_Renewed_Premium'], errors='coerce')
All_Insurer_Portal

In [ ]:
Greater_Than_0_MOTOR_Health = All_Insurer_Portal.query('New_Renewed_Premium > 0')
Greater_Than_0_MOTOR_Health

In [ ]:
final_M_H = Greater_Than_0_MOTOR_Health[Greater_Than_0_MOTOR_Health['New_Booking/Issued Date'].between('2022-07-01','2022-09-30' ,inclusive = True)].copy()
final_M_H

In [ ]:
final_M_H.info()

In [ ]:
final_M_H.drop(['Old Policy No.','Policy No.'], axis=1, inplace=True)
final_M_H

In [ ]:
from datetime import datetime

date = datetime.now().strftime("%m_%d")

In [ ]:
final_M_H.to_csv(f"0_{date}_All_Insurer_MO_He_Portal_data.csv",index=False)

### duplicated

In [ ]:
#All_Insurer_Portal.apply(lambda x: x.duplicated().sum()) # Duplicate